In [9]:
example_input = {
    "dependency_heads": [3,3,0,7,6,7,3,9,7,11,7,15,15,15,11,18,18,15,3],
    "dependency_relations": ["nsubj","advmod","root","mark","det","nsubj","ccomp","amod","obj","mark","advcl","det","amod","amod","obj","case","compound","nmod","punct"],
    "lemmas": ["it","also","recommend","that","the","authority","take","appropriate","measure","to","meet","the","specific","educational","need","of","Roma","child","."],
    "pos_tags": ["PRON","ADV","VERB","SCONJ","DET","NOUN","VERB","ADJ","NOUN","PART","VERB","DET","ADJ","ADJ","NOUN","ADP","PROPN","NOUN","PUNCT"],
    "predicates": ["_","_","PROPOSE","_","_","_","CARRY-OUT-ACTION","_","_","_","SATISFY_FULFILL","_","_","_","_","_","_","_","_"], 
    "words": [
        "It","also","recommends","that","the","authorities","take","appropriate","measures","to",
        "meet","the","specific","educational","needs","of","Roma","children","."],
    "roles": {
        "2": ["_","agent","_","topic","_","_","_","_","_","_","_","_","_","_","_","_","_","_","_"],
        "6": ["_","_","_","_","_","agent","_","_","patient","goal","_","_","_","_","_","_","_","_","_"],
        "10": ["_","_","_","_","_","_","_","_","_","_","_","_","_","_","theme","_","_","_","_"]
    }
}

In [10]:
sample_text = ' '.join(example_input['words'])
sample_text

'It also recommends that the authorities take appropriate measures to meet the specific educational needs of Roma children .'

In [44]:
import requests
http_url = 'https://verbatlas.org/api/model'
http_input = [{'text':sample_text, 'lang':'EN'}]
x = requests.post(http_url, json = http_input)
print(x)

<Response [200]>


In [36]:
import json
x_json = json.loads(x.text)
x_json

[{'tokens': [{'index': 0, 'rawText': 'It'},
   {'index': 1, 'rawText': 'also'},
   {'index': 2, 'rawText': 'recommends'},
   {'index': 3, 'rawText': 'that'},
   {'index': 4, 'rawText': 'the'},
   {'index': 5, 'rawText': 'authorities'},
   {'index': 6, 'rawText': 'take'},
   {'index': 7, 'rawText': 'appropriate'},
   {'index': 8, 'rawText': 'measures'},
   {'index': 9, 'rawText': 'to'},
   {'index': 10, 'rawText': 'meet'},
   {'index': 11, 'rawText': 'the'},
   {'index': 12, 'rawText': 'specific'},
   {'index': 13, 'rawText': 'educational'},
   {'index': 14, 'rawText': 'needs'},
   {'index': 15, 'rawText': 'of'},
   {'index': 16, 'rawText': 'Roma'},
   {'index': 17, 'rawText': 'children'},
   {'index': 18, 'rawText': '.'}],
  'annotations': [{'tokenIndex': 2,
    'verbatlas': {'frameName': 'PROPOSE',
     'roles': [{'role': 'Agent', 'score': 1.0, 'span': [0, 1]},
      {'role': 'Topic', 'score': 1.0, 'span': [3, 18]}]},
    'englishPropbank': {'frameName': 'recommend.01',
     'roles': 

In [10]:
import json
def format_response(response_text):
    null_tag = '_'
    phrases_formatted = []
    x_json = json.loads(response_text) if type(response_text) is str else response_text # response.text
    
    for phrase in x_json:
        x_formatted = {'roles':{}}
        # words
        x_formatted['words'] = [e['rawText'] for e in phrase['tokens']]
        # predicates + roles
        x_formatted['predicates'] = [null_tag]*len(phrase['tokens'])
        for annotation in phrase['annotations']:
            i = annotation['tokenIndex']
            if annotation['verbatlas']['frameName'] != null_tag: #TODO: roles!
                x_formatted['predicates'][i] = annotation['verbatlas']['frameName']

                x_formatted['roles'][i] = [null_tag]*len(phrase['tokens'])

                for j, role_for_i in enumerate(annotation['verbatlas']['roles']):
                    span_pos = role_for_i['span'][1 if j==0 else 0]
                    x_formatted['roles'][i][span_pos] = role_for_i['role'].lower()

        # in the end, append!
        phrases_formatted.append(x_formatted)

    return phrases_formatted

print(example_input)
print('\n')
print( format_response(x.text) )

{'dependency_heads': [3, 3, 0, 7, 6, 7, 3, 9, 7, 11, 7, 15, 15, 15, 11, 18, 18, 15, 3], 'dependency_relations': ['nsubj', 'advmod', 'root', 'mark', 'det', 'nsubj', 'ccomp', 'amod', 'obj', 'mark', 'advcl', 'det', 'amod', 'amod', 'obj', 'case', 'compound', 'nmod', 'punct'], 'lemmas': ['it', 'also', 'recommend', 'that', 'the', 'authority', 'take', 'appropriate', 'measure', 'to', 'meet', 'the', 'specific', 'educational', 'need', 'of', 'Roma', 'child', '.'], 'pos_tags': ['PRON', 'ADV', 'VERB', 'SCONJ', 'DET', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'PART', 'VERB', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'PROPN', 'NOUN', 'PUNCT'], 'predicates': ['_', '_', 'PROPOSE', '_', '_', '_', 'CARRY-OUT-ACTION', '_', '_', '_', 'SATISFY_FULFILL', '_', '_', '_', '_', '_', '_', '_', '_'], 'words': ['It', 'also', 'recommends', 'that', 'the', 'authorities', 'take', 'appropriate', 'measures', 'to', 'meet', 'the', 'specific', 'educational', 'needs', 'of', 'Roma', 'children', '.'], 'roles': {'2': ['_', 'agent', '_', 'topic'

In [103]:
type([1,2,3]) == list

True

# OTHER

In [322]:
import json
import requests
from copy import deepcopy
import re
from tqdm import tqdm

def load_data(lang_data_path):
    f = open(lang_data_path)
    res = json.load( f )
    f.close()
    return res

def list_to_phrase(l_w):
    phrase = ' '.join(l_w) if type(l_w) == list else l_w
    phrase = re.sub(r'( \')','\'',phrase)
    phrase = re.sub(r'( ’)','’',phrase)
    phrase = re.sub(r'(\b\. )|(\b\.$)',' . ',phrase)
    return phrase

def reconstruct_subwords(text_str_input, verbatlas_out, amuse_out):
    va_l = [ e['rawText'] for e in verbatlas_out['tokens'] ]
    am_l = [ e['text'] for e in amuse_out['tokens'] ]

    for i in range(len(verbatlas_out['annotations'])):
        l_k = list(verbatlas_out['annotations'][i].keys())
        for k in l_k:
            if k != 'tokenIndex' and k != 'verbatlas':
                verbatlas_out['annotations'][i].pop(k,None)

    if len(va_l) == len(am_l):
        return text_str_input, verbatlas_out, amuse_out
    
    text_l = text_str_input.split(' ')
    r_v_i = deepcopy(verbatlas_out)
    r_a_i = deepcopy(amuse_out)

    # verbatlas
    text_i = 0; json_i = 0
    while json_i < len(r_v_i['tokens']):

        edit_v = r_v_i['tokens'][json_i]['rawText'].lower() != text_l[text_i].lower()

        if not edit_v:
            text_i += 1; json_i += 1
            continue

        # for the predicates and roles

        for ii in range(len(r_v_i['annotations'])):
            r_v_i['annotations'][ii]['tokenIndex'] = r_v_i['annotations'][ii]['tokenIndex'] - 1 \
            if r_v_i['annotations'][ii]['tokenIndex'] > json_i else r_v_i['annotations'][ii]['tokenIndex']
            for iii in range(len(r_v_i['annotations'][ii]['verbatlas']['roles'])):
                for iiii in [0,1]:
                    r_v_i['annotations'][ii]['verbatlas']['roles'][iii]['span'][iiii] = \
                    r_v_i['annotations'][ii]['verbatlas']['roles'][iii]['span'][iiii] -1 \
                    if r_v_i['annotations'][ii]['verbatlas']['roles'][iii]['span'][iiii] > json_i \
                    else r_v_i['annotations'][ii]['verbatlas']['roles'][iii]['span'][iiii]

        r_v_i['tokens'][json_i]['rawText'] += r_v_i['tokens'][json_i+1]['rawText']

        for ii in range( json_i+2 , len(r_v_i['tokens']) ):
            r_v_i['tokens'][ii]['index'] -= 1

        del r_v_i['tokens'][json_i+1]

    # amuse
    text_i = 0; json_i = 0
    while json_i < len(r_a_i['tokens']):

        edit_a = r_a_i['tokens'][json_i]['text'].lower() != text_l[text_i].lower()

        if not edit_a:
            text_i += 1; json_i += 1
            continue

        r_a_i['tokens'][json_i]['text'] += r_a_i['tokens'][json_i+1]['text']

        for ii in range( json_i+2 , len(r_a_i['tokens']) ):
            r_a_i['tokens'][ii]['index'] -= 1

        del r_a_i['tokens'][json_i+1]
            
                        
    return text_str_input, r_v_i, r_a_i
    

def retrieve_data(data, verbatlas_link, amuse_link, lang="EN", chunk_dim=1):
    null_tag = '_'
    roles_precedence = [
        'NOUN','PROPN','ADP','PART','SCONJ','PRON','VERB','NUM',
        'ADJ','ADV','DET','PUNCT','SYM','AUX','X','CCONJ',
        'SPACE','INTJ', # added
    ]
    result = {}

    sentences_skipped = 0
    to_send = []

    pbar = tqdm(enumerate(data.items()))

    for data_i, (data_sample_key,data_sample_value) in pbar:

        text_sampled = list_to_phrase(data_sample_value['words'])

        to_send.append({ 'text': text_sampled, 'lang':lang, 'key_id':data_sample_key })
        if len(to_send)%chunk_dim != 0:
            continue
        
        pbar.set_description(f'percentage: {((data_i+1)/len(data)):.4f} (index={data_i})')

        res_v = requests.post(verbatlas_link, json = to_send)
        res_a = requests.post(amuse_link, json = to_send)
        status_code_good = res_v.status_code == 200 and res_a.status_code == 200

        if not status_code_good:
            print(f'verbatlas={res_v.status_code} | amuse={res_a.status_code} | skipping chunk!')
            sentences_skipped += len(to_send)
            to_send = []

        res_v = json.loads(res_v.text)
        res_a = json.loads(res_a.text)
        for t_i, r_v_i, r_a_i in zip(to_send, res_v, res_a):
            
            ti_text, r_v_i, r_a_i = reconstruct_subwords(t_i['text'], r_v_i, r_a_i)
            t_i['text'] = ti_text

            k_id = t_i['key_id']

            result[k_id] = {'roles':{}}
            # pos_tags
            result[k_id]['pos_tags'] = [ e['pos'] for e in r_a_i['tokens'] ]
            # lemmas
            result[k_id]['lemmas'] = [ e['lemma'] for e in r_a_i['tokens'] ]
            # words
            result[k_id]['words'] = [ e['rawText'] for e in r_v_i['tokens'] ]

            try:
                assert len(result[k_id]['lemmas']) == len(result[k_id]['words'])
            except:
                print(t_i['text'])
                print([ e['text'] for e in r_a_i['tokens'] ])
                print(result[k_id]['words'])
                raise Exception('error.')

            # predicates + roles
            result[k_id]['predicates'] = [null_tag]*len(r_v_i['tokens'])
            for annotation in r_v_i['annotations']:
                tokenIndex = annotation['tokenIndex']
                if annotation['verbatlas']['frameName'] != null_tag: # TODO: roles!
                    result[k_id]['predicates'][tokenIndex] = annotation['verbatlas']['frameName']

                    result[k_id]['roles'][str(tokenIndex)] = [null_tag]*len(r_v_i['tokens'])

                    for _, role_for_i in enumerate(annotation['verbatlas']['roles']):
                        span_pos = role_for_i['span']
                        
                        # TODO: use span!
                        pos_best_value = 999 # the less the better
                        best_index = span_pos[0]
                        for span_pos_i in range(span_pos[0],span_pos[1]+1):
                            try:
                                r_a_i['tokens'][span_pos_i]['pos']
                            except:
                                continue
                            if r_a_i['tokens'][span_pos_i]['pos'] in roles_precedence:
                                pos_value = roles_precedence.index( r_a_i['tokens'][span_pos_i]['pos'] )
                            else:
                                print('Warn: pos value not present:',r_a_i['tokens'][span_pos_i]['pos'])
                                pos_value = 999
                            if pos_value < pos_best_value:
                                pos_best_value = pos_value
                                best_index = span_pos_i
                                
                        result[k_id]['roles'][str(tokenIndex)][best_index] = role_for_i['role'].lower()
                
        to_send = []
    
    print(f'skipped {sentences_skipped} sentences out of {len(data.values())}')
    return result

In [323]:
verbatlas_url = 'http://127.0.0.1:3001/api/model'
amuse_url = 'http://127.0.0.1:3002/api/model'

In [324]:
test_str = "We urge the parties to refrain from the use of force and provocative acts , which only serve to undermine the peace process , and we appeal to them to take immediate steps to create the necessary environment for a restoration of peace , stability and the continuation of talks leading to a comprehensive , just and lasting peace based on Security Council resolutions 242 ( 1967 ) and 338 ( 1973 ) ."
test_str = list_to_phrase(test_str)
print(test_str)

ee = json.loads(requests.post(amuse_url, json = [{'text':test_str,'lang':'EN'}]).text)[0]
aa = json.loads(requests.post(verbatlas_url, json = [{'text':test_str,'lang':'EN'}]).text)[0]

rt, rv, ra = reconstruct_subwords(test_str, aa, ee)

We urge the parties to refrain from the use of force and provocative acts , which only serve to undermine the peace process , and we appeal to them to take immediate steps to create the necessary environment for a restoration of peace , stability and the continuation of talks leading to a comprehensive , just and lasting peace based on Security Council resolutions 242 ( 1967 ) and 338 ( 1973 ) .


In [325]:
errorrr = 'Mr . Lozinsky ( Russian Federation ) said that his delegation had taken note of the positive evaluation by the International Civil Service Commission ( ICSC ) of the activities of the Working Group on the Framework for Human Resources Management .'

vvvvv = json.loads(requests.post(verbatlas_url, json = [{'text':errorrr,'lang':'EN'}]).text)


In [326]:
en_data = load_data('../../../../data/EN/train.json')

In [328]:
en_to_send = retrieve_data(en_data, verbatlas_url, amuse_url, lang="EN")

percentage: 0.0434 (index=238): : 238it [01:11,  3.64it/s]

verbatlas=500 | amuse=200 | skipping chunk!


percentage: 0.2312 (index=1271): : 1271it [06:00,  2.59it/s]

verbatlas=500 | amuse=200 | skipping chunk!


percentage: 0.2516 (index=1383): : 1382it [06:31,  5.01it/s]

...

verbatlas=500 | amuse=200 | skipping chunk!


percentage: 0.9502 (index=5226): : 5225it [25:10,  3.97it/s]

verbatlas=500 | amuse=200 | skipping chunk!


percentage: 1.0000 (index=5500): : 5501it [26:42,  3.43it/s]

skipped 11 sentences out of 5501


In [330]:
import json

with open('../../../../data/EN/train_autogenerated.json', 'w') as outfile:
    json.dump(en_to_send, outfile, indent=4)

In [34]:
list(en_data.values())[0]

{'dependency_heads': [3, 3, 0, 8, 7, 7, 8, 3, 10, 8, 13, 13, 10, 3],
 'dependency_relations': ['nsubj',
  'advmod',
  'root',
  'mark',
  'det',
  'amod',
  'nsubj',
  'ccomp',
  'amod',
  'obj',
  'case',
  'amod',
  'nmod',
  'punct'],
 'lemmas': ['member',
  'also',
  'ask',
  'whether',
  'all',
  'social',
  'group',
  'enjoy',
  'equal',
  'access',
  'to',
  'higher',
  'education',
  '.'],
 'pos_tags': ['NOUN',
  'ADV',
  'VERB',
  'SCONJ',
  'DET',
  'ADJ',
  'NOUN',
  'VERB',
  'ADJ',
  'NOUN',
  'ADP',
  'ADJ',
  'NOUN',
  'PUNCT'],
 'predicates': ['_',
  '_',
  'ASK_REQUEST',
  '_',
  '_',
  '_',
  '_',
  'BENEFIT_EXPLOIT',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_'],
 'roles': {'2': ['agent',
   '_',
   '_',
   'theme',
   '_',
   '_',
   '_',
   '_',
   '_',
   '_',
   '_',
   '_',
   '_',
   '_'],
  '7': ['_',
   '_',
   '_',
   '_',
   '_',
   '_',
   'beneficiary',
   '_',
   '_',
   'theme',
   '_',
   '_',
   '_',
   '_']},
 'words': ['Members',
  'also',
  'asked',
  

In [33]:
list(en_to_send.values())[0]

{'roles': {2: ['agent',
   '_',
   '_',
   '_',
   '_',
   '_',
   'theme',
   '_',
   '_',
   '_',
   '_',
   '_',
   '_',
   '_'],
  7: ['_',
   '_',
   '_',
   '_',
   '_',
   '_',
   'experiencer',
   '_',
   '_',
   'stimulus',
   '_',
   '_',
   '_',
   '_']},
 'words': ['Members',
  'also',
  'asked',
  'whether',
  'all',
  'social',
  'groups',
  'enjoyed',
  'equal',
  'access',
  'to',
  'higher',
  'education',
  '.'],
 'predicates': ['_',
  '_',
  'ASK_REQUEST',
  '_',
  '_',
  '_',
  '_',
  'ENJOY',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_']}

In [17]:
en_data_sample = list(en_data.values())

In [18]:
from collections import Counter

pos_counter = Counter()

for e in en_data_sample:
    if 'roles' in e:
        # print(e['words'])
        for i, r in e['roles'].items():
            idx = int(i)
            # print(f"predicate = {e['predicates'][idx]}")
            for r_i, r_v in enumerate(r):
                if r_v != '_':
                    pos_tag = e['pos_tags'][r_i]
                    # print(f"role_word = {e['words'][r_i]} | role = {r_v} | pos = {pos_tag}")
                    pos_counter[pos_tag] += 1
    # print('\n')

In [17]:
out_verbatlas = requests.post(verbatlas_url, json = en_to_send)

In [ ]:
out_amuse = requests.post(amuse_url, json = en_to_send)